In [ ]:
cd("/Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/spinfoam-critical-pts")

In [ ]:
using LinearAlgebra, Combinatorics
using Printf
using Dates

include("GeometryTypes.jl")          # defines GeometryDataset, GeometryCollection
#include("load_geometry_data.jl")        # low-level parsers FIRST
#include("GeometryDataLoader.jl")        # defines GeometryDataset
include("write_geometry_data.jl")
# --- include all modules ---
include("SimplexGeometry.jl")
include("SpinAlgebra.jl")
include("volume.jl")
include("TetraNormals.jl")
include("DihedralAngles.jl")
include("LorentzGroup.jl")
include("ThreeDTetra.jl")
include("Su2Su11FromBivector.jl")
include("XiFromSU.jl")
include("FaceNormals3D.jl")
include("KappaFromNormals.jl")
include("GeometryConsistency.jl")
include("GeometryPipeline.jl")
include("KappaOrientation.jl")
include("FourSimplexConnectivity.jl")
include("write_connectivity.jl")
include("FaceXiMatching.jl")
include("FaceMatchingChecks.jl")


using .GeometryTypes: GeometryDataset, GeometryCollection
using .GeometryPipeline: run_geometry_pipeline
using .GeometryConsistency: check_sl2c_parallel_transport, check_so13_parallel_transport, check_closure_bivectors
using .KappaOrientation: fix_kappa_signs!
using .FourSimplexConnectivity: build_global_connectivity
using .write_connectivity_data: save_connectivity_data
using .FaceXiMatching: run_face_xi_matching
using .FaceMatchingChecks: check_all

# for mkpath
import Base.Filesystem: mkpath

In [ ]:
bdypoints_all = [
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 1.0, 1.0],
        [0.0, 1.0, 1.0, 1.0],
        [0.5, 1.0, 1.0, 1.0]
    ],
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 1.0, 1.0],
        [0.5, 1.0, 1.0, 1.0],
        [0.0, 1.0, 0.0, 1.0]
    ]
]

In [ ]:
geom = nothing
datasets = GeometryDataset[]
ns = length(bdypoints_all)

base_folder = "geom_output/new-data/"
mkpath(base_folder)

for s in 1:ns
    subfolder = base_folder * "simplex$(s)/"
    mkpath(subfolder)

    bdypoints = bdypoints_all[s]

    println("Boundary points for simplex $s:")

    ds = run_geometry_pipeline(bdypoints, subfolder)  # GeometryDataset
    push!(datasets, ds)

end

In [ ]:
geom = GeometryCollection(datasets);

four_simplices = [[1,2,3,4,5], [1,2,4,5,6]];

fix_kappa_signs!(four_simplices, geom);

build_global_connectivity(four_simplices, geom);

In [ ]:
conn = build_global_connectivity(four_simplices, geom)
push!(geom.connectivity, conn);

In [ ]:
run_face_xi_matching(geom);

In [ ]:
check_all(geom)

In [ ]:
include("GaugeFixing.jl")
using .GaugeFixingSU

In [ ]:
sl2ctest2 = [geom.simplex[i].solgsl2c for i in 1:ns]
ntet = length(sl2ctest2[1])
sharedTetsPos = geom.connectivity[1]["sharedTetsPos"]
sgndet = [geom.simplex[i].sgndet for i in 1:ns]

# --- SU(2) gauge fixing on spacelike tets ---
GaugeFixUpperTriangle, oppositesl2c = GaugeFixingSU.build_gauge_fix_sets(sharedTetsPos, sgndet)
su2triangle = GaugeFixingSU.build_su2_triangle(sl2ctest2, GaugeFixUpperTriangle, oppositesl2c);

In [ ]:
sl2c3 = [
        [ sl2ctest2[i][j] * inv(su2triangle[i][j]) for j in 1:ntet ]
        for i in 1:ns
    ];

In [ ]:
bdybivec55stest2 = [geom.simplex[i].bdybivec55 for i in 1:ns]
bivec3 = [
        [ [ su2triangle[i][j] * B * inv(su2triangle[i][j])
            for B in bdybivec55stest2[i][j] ]
          for j in 1:ntet ]
        for i in 1:ns
    ];

In [ ]:
tetareasign = [geom.simplex[i].tetareasign for i in 1:ns]
tetn0_3 = FaceXiMatching.compute_tetn0signtest(bivec3, sgndet, tetareasign);

In [ ]:
su3     = FaceXiMatching.build_su_from_bivec(bivec3, sgndet, tetareasign, tetn0_3)
xi3     = FaceXiMatching.build_xi_from_su(su3, sgndet, tetareasign, tetn0_3);

In [ ]:
xi4 = GaugeFixingSU.normalize_timelike_xi(xi3, tetareasign);

In [ ]:
timelike_pairs, gaugespacelike, gaugetimelike, lookup = GaugeFixingSU.build_timelike_data(sharedTetsPos, sgndet, tetareasign);


In [ ]:
Uinverse = GaugeFixingSU.build_Uinverse(ns, ntet, lookup, gaugespacelike, su3);

In [ ]:
xi5      = GaugeFixingSU.apply_left_on_xi(Uinverse, xi4);

In [ ]:
xi6      = GaugeFixingSU.normalize_timelike_xi(xi5, tetareasign);

In [ ]:
U2       = GaugeFixingSU.build_U2(ns, ntet, lookup, gaugetimelike, xi6);

In [ ]:
xi7      = GaugeFixingSU.apply_left_on_xi(U2, xi6)
xi_final = GaugeFixingSU.normalize_timelike_xi(xi7, tetareasign);

In [ ]:
sl2c4 = [
        [ sl2c3[i][j] * inv(Uinverse[i][j]) * inv(U2[i][j]) for j in 1:ntet ]
        for i in 1:ns
    ];

In [ ]:
xisoln = GaugeFixingSU.compute_xisoln(xi_final, sgndet, tetareasign, tetn0_3)

In [ ]:
xisoln[1][2]

In [ ]:

gdataof = GaugeFixingSU.compute_gdataof(sl2c4)

In [ ]:

zdataf = GaugeFixingSU.compute_zdataf(
        tetareasign,    # you use kappa later, check order!
        tetareasign,
        gdataof,
        xi_final
    )